# ACS2 in Heist environment

In [2]:
from __future__ import unicode_literals
%matplotlib inline
import os
import sys
sys.path.append(r'C:\\Users\\gosia\\OneDrive\\Pulpit\\mgr\\mgr\\heist_autoencoder')
sys.path.append(r'C:\\Users\\gosia\\OneDrive\\Pulpit\\mgr\\mgr\\heist_autoencoder\\utils')

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from pylab import cm
import pathlib
import numpy as np
import pandas as pd
import logging
logging.basicConfig(level=logging.WARN)
from lcs import Perception
from lcs.agents import EnvironmentAdapter
from lcs.agents.acs2 import ACS2, Configuration, ClassifiersList
from lcs.metrics import population_metrics
import gym

In [3]:
from utils.cache import get_from_cache_or_run


In [4]:
# Ścieżka do folderu zawierającego skrypty
scripts_folder_path = r'C:\\Users\\gosia\\OneDrive\\Pulpit\\mgr\\mgr\\experiments\\utils'

# Zmiana bieżącego katalogu na katalog zawierający skrypty
os.chdir(scripts_folder_path)

%run experiments.py
%run metrics.py
%run plots.py

ModuleNotFoundError: No module named 'pyalcs'

In [5]:
cache_prefix = f'{pathlib.Path().absolute()}/cache/'

## Heist Environment

In [6]:
env_maxpool = gym.make('heist_autoencoder.gym_autoencoder.heist.envs:heist-auto-maxpool-v0')
observation = env_maxpool.reset()
print("Observation:", observation)
env_maxpool.render()

UnregisteredEnv: No registered env with id: gym_autoencoder.heist.envs:heist-auto-maxpool-v0

In [ ]:
env_maxpoolbig = gym.make('gym_autoencoder.heist.envs:heist-auto-maxpool-big-v0')
observation = env_maxpoolbig.reset()
print("Observation:", observation)
env_maxpoolbig.render()

Observation: [ 0.         18.89122     0.          0.          0.          0.
 13.824522    1.2203252  13.588886   21.643892    7.669902    0.
 11.082521    5.0379176   0.         10.410666    4.9378753   5.5152864
  7.6412516   7.487723    0.          2.953337    9.97312     3.3854911
  9.393656    0.          0.          0.          0.          0.04432361
  0.         11.889985  ]


In [ ]:
env_vaealex = gym.make('gym_autoencoder.heist.envs:heist-vae-alex-v0')
env_vaealex.reset()
print("Observation:", observation)
env_vaealex.render()

Observation: [ 0.         18.89122     0.          0.          0.          0.
 13.824522    1.2203252  13.588886   21.643892    7.669902    0.
 11.082521    5.0379176   0.         10.410666    4.9378753   5.5152864
  7.6412516   7.487723    0.          2.953337    9.97312     3.3854911
  9.393656    0.          0.          0.          0.          0.04432361
  0.         11.889985  ]


## Environment setup

In [ ]:
_range, _low = (env_maxpoolbig.observation_space.high - env_maxpoolbig.observation_space.low, env_maxpoolbig.observation_space.low)

class DiscretizedWrapper(gym.ObservationWrapper):
    def discretize(self, obs, _type=int):
        print("_low:", _low)
        print("_range:", _range)
        print("obs:", obs)
        
        r = (obs + np.abs(_low)) / _range
        print("r:", r)
        
        b = (r * 10).astype(int)
        print("b:", b)
        
        discretized_obs = b.astype(_type).tolist()
        print("discretized_obs:", discretized_obs)
        
        return discretized_obs
    def observation(self, obs):
        return self.discretize(obs, _type=int)

In [ ]:
env_maxpool = DiscretizedWrapper(env_maxpool)
env_maxpoolbig = DiscretizedWrapper(env_maxpoolbig)
env_vaealex = DiscretizedWrapper(env_vaealex)

## Basic Metrics

In [ ]:
learning_rate = 0.2
discount_factor = 0.95
epsilon = 0.2
zeta = 0.0001

basic_cfg = {
    'perception_bits': 32,
    'possible_actions': 9,
    'do_ga': False,
    'beta': learning_rate,
    'epsilon': epsilon,
    'gamma': discount_factor,
    'zeta': zeta,
    'user_metrics_collector_fcn': common_metrics,
    'biased_exploration_prob': 0,
    'metrics_trial_freq': 1
}

trials = 4000
experiments = 5

In [ ]:
result = get_from_cache_or_run(r'C:\\Users\\gosia\\OneDrive\\Pulpit\\mgr\\mgr\\experiments\\cache\\4000trials-maxpool-maxpoolbig-vaealex-5exp.dill', avg_experiments(experiments, envs={ "MaxPool": env_maxpool,"MaxPoolBig": env_maxpoolbig, "VAEAlex": env_vaealex}, trials=trials, params=basic_cfg,fun=run_experiments_alternating))

_low: [-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf]
_range: [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf]
obs: [0.         0.         0.         0.         0.         0.
 2.9779801  3.5367608  0.         0.         0.         1.047847
 0.5763031  0.         0.         1.4987997  0.         0.
 0.9963366  0.         0.         0.         0.         0.21062973
 3.3113694  1.8796037  2.2925544  0.         0.         0.
 1.6668147  2.9113538 ]
r: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
b: [-2147483648 -2147483648 -2147483648 -2147483648 -2147483648 -2147483648
 -2147483648 -2147483648 -2147483648 -2147483648 -2147483648 -2147483648
 -2147483648 -2147483648 -2147483648 -2147483648 -2147483648 -21474836

C:\Users\gosia\AppData\Local\Temp\ipykernel_4920\1885347072.py:9: RuntimeWarning: invalid value encountered in divide
  r = (obs + np.abs(_low)) / _range
C:\Users\gosia\AppData\Local\Temp\ipykernel_4920\1885347072.py:12: RuntimeWarning: invalid value encountered in cast
  b = (r * 10).astype(int)


TypeError: TypedList.__init__() got multiple values for argument 'oktypes'

: 